In [74]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from scipy.special import gamma

In [75]:
df = pd.read_csv('dataset/test/manual_data.csv')
n = len(df)
k = round((n**(3/8))+1)
# df.pop('id')
# result1 = pd.read_csv('dataset/test/result1.csv')
# result2 = pd.read_csv('dataset/test/result2.csv')

# score1 = silhouette_score(df,result1)
# score2 = silhouette_score(df,result2)

In [76]:
scaler = StandardScaler()
scale = scaler.fit_transform(df)

In [77]:
model = NearestNeighbors(n_neighbors=k, metric='euclidean')

# Fit the model to the data
model.fit(scale)

# Find the k nearest neighbors for each point in X
d, i = model.kneighbors(scale)

In [78]:
distances = d[:, 1:]
indices = i[:, 1:]
further_n = [row[-1] for row in d]
further_n = np.array(further_n)

In [79]:
Vx = np.prod(distances, axis=1)
# Vx = np.pi**((k-1)/2) / gamma((k-1)/2 + 1) * further_n**(k-1)

mulByK = distances * (k-1)
# Hx = np.pi**((k-1)/2) / gamma((k-1)/2 + 1) * (further_n*(k-1))**(k-1)
Hx = mulByK.sum(axis=1)

onePernVx = 1 / (n * Vx)

de = 1 - (distances / Hx.reshape(n,1))
sum = Vx * (de.sum(axis=1))

In [80]:
def triangular_kernel(x):
    return np.maximum(0, 1 - np.abs(x))

In [81]:
px = np.array([triangular_kernel(x) for x in sum])

In [82]:
sum

array([0.19985614, 0.00677012, 2.66055689, 0.31410041, 0.05941634,
       0.1385984 , 0.36065457, 0.44896301, 0.28797203, 1.74158583,
       0.31453493, 0.00444742, 0.11449055, 0.19283202, 0.10991041,
       0.00905546, 0.49453747, 4.0780326 , 0.02464206, 0.2585936 ])

In [83]:
# Create an array of zeros with the same shape as indices
pxMatrix = np.zeros(indices.shape)

# Loop through the indices array and replace with the corresponding value
for i in range(indices.shape[0]):
    for j in range(indices.shape[1]):
        pxMatrix[i, j] = px[indices[i, j]]

print(pxMatrix)

[[0.7414064  0.94058366 0.99094454]
 [0.99555258 0.99094454 0.97535794]
 [0.         0.         0.88550945]
 [0.63934543 0.80014386 0.7414064 ]
 [0.99322988 0.99555258 0.99094454]
 [0.80716798 0.50546253 0.89008959]
 [0.68589959 0.80014386 0.7414064 ]
 [0.68546507 0.94058366 0.80014386]
 [0.7414064  0.97535794 0.80014386]
 [0.         0.         0.88550945]
 [0.55103699 0.80014386 0.63934543]
 [0.99322988 0.99094454 0.97535794]
 [0.89008959 0.50546253 0.8614016 ]
 [0.8614016  0.50546253 0.89008959]
 [0.88550945 0.50546253 0.8614016 ]
 [0.99555258 0.99322988 0.97535794]
 [0.8614016  0.80716798 0.89008959]
 [0.         0.         0.88550945]
 [0.99094454 0.99322988 0.99555258]
 [0.80014386 0.71202797 0.68589959]]


In [84]:
mx = indices + 1

In [85]:
max_index_values = []

# Loop through each sub-array in the value array
for i, sub_array in enumerate(pxMatrix):
    # Find the index of the maximal value in the sub-array
    max_index = np.argmax(sub_array)
    # Get the corresponding index from the index array
    max_index_value = mx[i, max_index]
    # Append the max_index_value to the new array
    max_index_values.append([i + 1, max_index_value])

# Print the result
print(max_index_values)

[[1, 16], [2, 12], [3, 13], [4, 1], [5, 12], [6, 15], [7, 1], [8, 5], [9, 19], [10, 13], [11, 1], [12, 2], [13, 15], [14, 15], [15, 13], [16, 12], [17, 15], [18, 13], [19, 12], [20, 1]]


In [86]:
def merge_subarrays(arrays):
    # Flatten the list of subarrays into a single list
    flattened = [item for sublist in arrays for item in sublist]

    # Repeat until there are no changes
    while True:
        # Create a set of unique values in the flattened list
        unique_values = set(flattened)

        # Merge subarrays that have the same members and remove duplicates
        new_arrays = []
        for value in unique_values:
            subarrays_with_value = [subarray for subarray in arrays if value in subarray]
            if len(subarrays_with_value) > 1:
                merged_subarray = list(set([item for sublist in subarrays_with_value for item in sublist]))
                new_arrays.append(merged_subarray)
            else:
                new_arrays.extend(subarrays_with_value)

        # Check if the arrays have changed and update the arrays if necessary
        if new_arrays == arrays:
            break
        arrays = new_arrays
        flattened = [item for sublist in arrays for item in sublist]

    return arrays


In [87]:
merged = merge_subarrays(max_index_values)

In [88]:
unique_tuples = set(tuple(sorted(merged[i])) for i in range(len(merged)))
unique_lst = [list(t) for t in unique_tuples]

print(unique_lst)


[[3, 6, 10, 13, 14, 15, 17, 18], [1, 2, 4, 5, 7, 8, 9, 11, 12, 16, 19, 20]]
